# Case
**Creates binary labels in raster format**

The labels are either based on a provided text- or Esri-shapefile.<br>
To spatially align the labels with other data sets, a template raster (base raster) can be provided.


In [3]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import os

from pathlib import Path
from typing import List, Tuple, Optional, Union
from rasterio.enums import MergeAlg
    
from rasterio import features

import sys
if sys.version_info < (3, 9):
    from importlib_resources import files
else:
    from importlib.resources import files

from beak.utilities.conversions import create_geodataframe_from_points, create_geodataframe_from_polygons, _rasterize_vector_process, create_binary_raster
from beak.utilities.io import save_raster, check_path, load_raster, load_dataset


# Load data

**User definitions**

In [18]:
# Set paths
BASE_PATH = files("beak.data")
PATH_SHAPEFILE = files("beak.data") / "BOUNDARIES" / "tl_2012_us_state_EPSG_4326.shp"
OUT_FILE = "test_base_raster_fn.tif"

# Reference coordinate system code
RESOLUTION = 0.025

QUERY = "NAME!='Alaska' & NAME!='Hawaii' & NAME!='Puerto Rico' & NAME!='United Sats Virgin Islands' & NAME!='American Samoa' & NAME!='Guam' & NAME!='Commonwealth of the Northern Mariana Islands'"


# Create Base Raster

## From shapefile

`fill_negatives`<p>
- True will fill the nodata values with a fill value, commonly 0 (i.e., the extent of the raster is filled completeley with values)<p>
- False will only convert the queried polygons to **one** and the rest of the raster will be **nodata**<p>
**Base rasters** commonly have **nodata** (`fill_negatives=False`) instead of filled values

In [21]:
# Load shapefile and create geodataframe
gdf = gpd.read_file(PATH_SHAPEFILE)

# Save raster
base_array = create_binary_raster(gdf, resolution=RESOLUTION, query=QUERY, all_touched=False, same_shape=True, fill_negatives=False, out_file=OUT_FILE)